# 点群の特性
点群は座標値を持つ点の集合であり、点群の点の一つ一つには座標値があります。この章では、この点群という表現の特性について、他の表現(主に画像)との比較も行いながら紹介、説明します。この章で説明する特徴は節ごとに分けてあります:

- 点群の点の情報
- 不規則な点の並び
  - 疎もしくは密な表現
  - 点間の隣接関係の定義
- 剛体変換に対する不変性 (タスクによる)
- 点の順不同に対する形状不変性

ここでは、これらを一つづつ紹介、説明していきます。
なお、コードも含まれますが、点群を視覚化するためであるため、コードの内容に注目する必要はありません。


## 点群の点の情報
点群の点は座標値以外に、色情報や法線などの情報を持ち合わせます。持ち合わせる情報としては以下の情報が挙げられます。

- **色 (RGB)**: 色情報はビュワーなどで点を表示する際に点に着色するために使用されたり、点群処理に利用されたりします。ただし、色情報の取得と点群情報(3次元座標情報)の取得はそれぞれ別のセンサーで行われるため、点に割り振られる色がずれることがあります。
- **法線**: 法線から立体オブジェクトの表面を推定したり(再構築タスクに最適)、平面検出を行ったりする際に利用されたりします。法線の取得方法の例として、点群から直接取得する、他の3D表現であるメッシュ表現から得る等が挙げられます。
- **反射強度**: LiDARなどのレーザーセンサーで得られる点ごとのレーザーの反射強度値。レーザーが当たったオブジェクトの反射率にも左右されるため、特徴抽出に使用されたりします。

注意点として、この情報は他の3D表現でも持ち合わせることがあります。


## 不規則な並び
点の位置は座標値で決まるため、点は画像のピクセルのように規則的な並びを持ちません。このような特性を持つことにより、いくつか留意することがあります。以下でその説明と関連するコードを示します。

本subsectionで使用するパッケージは以下のとおりです。

In [1]:
import numpy as np
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.io import Points as io

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 疎もしくは密な表現
ここで言う疎と密は、ある範囲内の点の密度のことを指します。この密度についてわかりやすい例として、センサーから取得した点群が当てはまります。実際のセンサーから取得したデータを下のコードの実行で確認できます。


In [2]:
# Read a point cloud
xyz, rgb, data = io.read('../data/kitti_sample.ply')

# Visualize a point cloud
obj_points = jv.point(xyz, rgb, color_range=[0, 255], point_size=0.3)
jv.display([obj_points])

Output()

上の出力はセンサーから点までの距離を色の変化によって示しています。赤がセンサーから近く、徐々に青になることでセンサーから距離が離れていることを示しています。ここで、センサーからの距離によって点がまばらになっていくことがわかると思います。
これは、距離が遠くなるにつれて、オクルージョンやセンサー自身の解像度などの複数の要因が影響してまばらなになっています(この要因についてはここでは詳細に説明しません)。

### 要素間の明確なつながりを持たない表現

点群は点の集合であり、点と点の明確な隣接関係を持ちません。ここで言う「隣接関係」とは、要素と要素のつながりのことを指します。  
例えば、グラフ表現であるなら、グラフの要素である頂点は辺というつながりによって他の頂点とのつながりを持ちます。画像であるなら、画像の要素であるピクセルは必ず隣にピクセルがあり、それとつながりを持ちます。  これらの表現に比べて、点群の点は他の点との明確な隣接関係があらかじめ設けられていません。

この隣接関係が明確でない特性は、問題を生むことがあります。例えば、隣接する要素を必要とする畳み込みアルゴリズムの適用を阻むという問題があります。これらの問題を解決するには以下のような方法が挙げられます。

1. 点群の各点に対してk近傍(kNN)やボールクエリ等を実施し、点の近くにある点との隣接関係を持たせる。
2. 点群をボクセル表現に変換する。

1番目の方法は、点の周りの情報を収集するためによく使われます。例えば、handcrafted featureを生成する際は、点の周りにある他の点の情報を参照するためにkNNが使用されます。また、深層学習手法などの畳込み機構を設けた処理では、同様に周りの点の特徴を畳み込むために使用されます。
2番目の方法は、点群を画像の様なグリッドに沿ったデータフォーマットに変換することができます。これにより、例えば深層学習では画像用の2D CNNを拡張した3D CNNの適用が可能になりました。


## 剛体変換に対する不変性 (タスクによる)
点群に対して剛体変換を用いることで点群が持つ形状を崩さずに姿勢の変更や平行移動が可能です。タスクによっては剛体変換された点群に対する処理結果が変化しないようにする必要があります。例えばオブジェクトに対する分類タスクでは、与えられる入力点群の姿勢が一定ではない可能性があり、一定でない場合はどの様な姿勢に対しても不変な予測を行う必要があります。
「タスクによる」とあるのは、姿勢や位置そのものを予測するタスク(例:姿勢推定)などが当てはまらないためです。これらのタスクは、与えられたデータの姿勢や位置に合わせて予測を行う必要があります。





## 点の順不同による形状不変性

点群は画像や文字列の様に要素に順序があるわけではないため、点の入力順序を変更したとしても点群が示す形状は変わりません。例えば、オブジェクト点群内にある点の順序をランダムに入れ替えたとしても、オブジェクトの形状は変わりません。
例として、オリジナルの点群と点の順序を入れ替えた点群をそれぞれ表示します。この表示で使うパッケージは以下のとおりです。

In [3]:
import numpy as np
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.io import Points as io

点の順序入れ替え前と後を表示するコードは以下のとおりです。X軸で0~1の間にあるウサギの点群が入れ替え前、1~2の間にあるウサギの点群が入れ替え後です。  
Note: 見比べを行うために入れ替え後の点群の位置を平行移動させています。  
問題点: 平行移動による形状が変わらないことへの言及がない

In [4]:
xyz, rgb, _ = io.read('../data/bunny_pc.ply')

idxs = np.random.choice(np.arange(len(xyz)), len(xyz), replace=False)
random_xyz = xyz[idxs]
random_rgb = rgb[idxs]

random_xyz[:, 0] += 1.0 # slide x
obj_points = jv.point(xyz, rgb)
obj_random_points = jv.point(random_xyz, random_rgb)
jv.display([obj_points, obj_random_points])

Output()

上の出力から見てわかるように、ウサギの形状も色も変わってないことが示されていると思います。  
実際にこれらの点群を処理する場合、その手法の出力は点群の点の順序に依存しない処理方法が必要となります。例えば機械学習の分野の分類タスクを解く場合、ある点群AとAの点の順序を変えた点群Bを機械学習モデルに入力したとしても、AとBの分類予測ラベルは同じでなければいけません。

# Reference

